## Import required libraries

In [1]:
import zipfile
import os
import sys
import pandas as pd
import openai
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
#!pip install azure-search-documents==11.4.0

In [ ]:
#!pip install langchain

In [ ]:
!pip freeze

In [2]:
from openai import AzureOpenAI
import json
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient, SearchIndexingBufferedSender  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryCaptionResult,
    QueryAnswerResult,
    SemanticErrorMode,
    SemanticErrorReason,
    SemanticSearchResultsType,
    QueryType,
    VectorizedQuery,
    VectorQuery,
    VectorFilterMode,    
)

In [3]:
from azure.search.documents.indexes.models import (  
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    SemanticPrioritizedFields,
    SemanticField,  
    SearchField,  
    SemanticSearch,
    VectorSearch,  
    HnswAlgorithmConfiguration,
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
    VectorSearchAlgorithmKind,
    VectorSearchProfile,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    VectorSearch,
    ExhaustiveKnnParameters,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    SemanticField,  
    SearchField,  
    VectorSearch,  
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
)  

In [4]:
# Configure environment variables  
service_endpoint = os.getenv("AZURE_AISEARCH_ENDPOINT") 
index_name = "finance-bench-small-sk" 
key = os.getenv("AZURE_AISEARCH_API_KEY") 
model: str = os.getenv("AZURE_OPENAI_EMBEDDINGS_MODEL_NAME") 
credential = AzureKeyCredential(key)

In [5]:
client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version = "2023-05-15",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

In [6]:
# Configure OpenAI settings
openai.api_type = os.getenv('AZURE_OPENAI_API_TYPE')
openai.api_base = os.getenv('AZURE_OPENAI_ENDPOINT')
openai.api_version = os.getenv('AZURE_OPENAI_API_VERSION')
openai.api_key = os.getenv('AZURE_OPENAI_API_KEY')
model: str = os.getenv('AZURE_OPENAI_EMBEDDINGS_MODEL_NAME') 

In [7]:
azure_search_endpoint: str = os.getenv('AZURE_AISEARCH_ENDPOINT') 
azure_search_key: str = os.getenv('AZURE_AISEARCH_API_KEY')
credential = AzureKeyCredential(azure_search_key)

In [8]:
from langchain.embeddings import AzureOpenAIEmbeddings
embeddings = AzureOpenAIEmbeddings(
        azure_deployment="text-embedding-ada-002",
        openai_api_version="2023-05-15",
        chunk_size=10,
    )

/home/sebacastillo/.genai3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.azure_openai.AzureOpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(


# Nuevo Índice

In [9]:
index_name: str = "finance-bench-small-sk"

## Perform a vector similarity search

This example shows a pure vector search using the vectorizable text query, all you need to do is pass in text and your vectorizer will handle the query vectorization.

In [10]:
def generate_embeddings(text, model=model):
        return client.embeddings.create(input = [text], model=model).data[0].embedding

In [11]:
# Pure Vector Search
query = "Revenue of Microsoft"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="Embedding")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["Text", "Id", ],
)  
  
for result in results:  
    print(f"Id: {result['Id']}")  
    print(f"Text: {result['Text']}")    
    print('-'*100)  


Id: MTg1
Text: manufacturing, marketing, and selling our other products and services; and income taxes.
Highlights from fiscal year 2023 compared with fiscal year 2022 included:
•
Microsoft Cloud revenue increased 22% to $111.6 billion.
•
Office Commercial products and cloud services revenue increased 10% driven by Office 365 Commercial growth of 13%.
•
Office Consumer products and cloud services revenue increased 2% and Microsoft 365 Consumer subscribers increased to 67.0 million.
•
LinkedIn revenue increased 10%.
•
Dynamics products and cloud services revenue increased 16% driven by Dynamics 365 growth of 24%.
•
Server products and cloud services revenue increased 19% driven by Azure and other cloud services growth of 29%.
•
Windows original equipment manufacturer licensing (“Windows OEM”) revenue decreased 25%.
•
Devices revenue decreased 24%.
•
Windows Commercial products and cloud services revenue increased 5%.
•
Xbox content and services revenue decreased 3%.
 
•
----------------

## Perform an Exhaustive KNN exact nearest neighbor search

This example shows how you can exhaustively search your vector index regardless of what index you have, HNSW or ExhaustiveKNN. You can use this to calculate the ground-truth values.

In [15]:
# Pure Vector Search  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="Embedding", exhaustive=True)
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["Text", "Id", ],
)  
  
for result in results:  
    print(f"Id: {result['Id']}")  
    print(f"Text: {result['Text']}")    
    print('-'*100)  




Id: MTg1
Text: manufacturing, marketing, and selling our other products and services; and income taxes.
Highlights from fiscal year 2023 compared with fiscal year 2022 included:
•
Microsoft Cloud revenue increased 22% to $111.6 billion.
•
Office Commercial products and cloud services revenue increased 10% driven by Office 365 Commercial growth of 13%.
•
Office Consumer products and cloud services revenue increased 2% and Microsoft 365 Consumer subscribers increased to 67.0 million.
•
LinkedIn revenue increased 10%.
•
Dynamics products and cloud services revenue increased 16% driven by Dynamics 365 growth of 24%.
•
Server products and cloud services revenue increased 19% driven by Azure and other cloud services growth of 29%.
•
Windows original equipment manufacturer licensing (“Windows OEM”) revenue decreased 25%.
•
Devices revenue decreased 24%.
•
Windows Commercial products and cloud services revenue increased 5%.
•
Xbox content and services revenue decreased 3%.
 
•
----------------

## Perform a Cross-Field Vector Search

This example shows a cross-field vector search that allows you to query multiple vector fields at the same time. Note, ensure that the same embedding model was used for the vector fields you decide to query.

In [ ]:
# # Pure Vector Search  
# search_client = SearchClient(service_endpoint, index_name, credential=credential)
# vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="contentVector, titleVector")
# # Use the below query to pass in the raw vector query instead of the query vectorization
# # vector_query = RawVectorQuery(vector=generate_embeddings(query), k=3, fields="titleVector, contentVector")
  
# results = search_client.search(  
#     search_text=None,  
#     vector_queries= [vector_query],
#     select=["company_name", "source", "doc_type", "page_content"],
# )  
  
# for result in results:  
#     print(f"Company_name: {result['company_name']}")  
#     print(f"Score: {result['@search.score']}")  
#     print(f"Source: {result['source']}")  
#     print(f"Doc_type: {result['doc_type']}")  
#     print(f"Page_content: {result['page_content']}\n")
#     print('-'*100)  



## Perform a Multi-Vector Search

This example shows a cross-field vector search that allows you to query multiple vector fields at the same time by passing in multiple query vectors. Note, in this case, you can pass in query vectors from two different embedding models to the corresponding vector fields in your index.

In [ ]:
# Multi-Vector Search
# query = "tools for software development"  
  
# search_client = SearchClient(service_endpoint, index_name, credential=credential)  
# vector_query_1 = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="titleVector")
# vector_query_2 = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="contentVector")
  
# results = search_client.search(  
#     search_text=None,  
#     vector_queries=[vector_query_1, vector_query_2],
#     select=["title", "content", "category"],
# )  
  
# for result in results:  
#     print(f"Title: {result['title']}")  
#     print(f"Score: {result['@search.score']}")  
#     print(f"Content: {result['content']}")  
#     print(f"Category: {result['category']}\n")  


## Perform a Pure Vector Search with a filter
This example shows how to apply filters on your index. Note, that you can choose whether you want to use Pre-Filtering (default) or Post-Filtering.

In [18]:
# Pure Vector Search
query = "What is the Revenue of 2019"  

search_client = SearchClient(service_endpoint, index_name, credential=credential)
  
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="Embedding")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    vector_filter_mode=VectorFilterMode.PRE_FILTER,
    filter="Description eq 'BESTBUY'",
    select=["Text", "Id", "Description", 'AdditionalMetadata', 'ExternalSourceName' ],
)  
  
for result in results:  
    print(f"Id: {result['Id']}")  
    print(f"Description: {result['Description']}")
    print(f"AdditionalMetadata: {result['AdditionalMetadata']}")
    print(f"ExternalSourceName: {result['ExternalSourceName']}")        
    print('-'*100)  



Id: MTQxNg==
Description: BESTBUY
AdditionalMetadata: 2019_10K_30_2482
ExternalSourceName: 
----------------------------------------------------------------------------------------------------
Id: MTQxMw==
Description: BESTBUY
AdditionalMetadata: 2019_10K_30_0
ExternalSourceName: 
----------------------------------------------------------------------------------------------------
Id: MTQyMA==
Description: BESTBUY
AdditionalMetadata: 2019_10K_31_2618
ExternalSourceName: 
----------------------------------------------------------------------------------------------------


## Perform a Hybrid Search

In [19]:
# Hybrid Search
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))  
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="Embedding")

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["Text", "Id", "Description" ],
    top=3
)  
  
for result in results:  
    print(f"Id: {result['Id']}")  
    print(f"Description: {result['Description']}")      
    print(f"Text: {result['Text']}")   
    print('-'*100)  

Id: MTQyMA==
Description: BESTBUY
Text: e in the excess profits. In fiscal 2019, we recognized $7 million of such profit-share revenue, with an equal impact to gross profit andoperating income. In fis
cal 2018, we recognized $59 million of such profit-share revenue. The fiscal 2019 profit-share revenue decrease from fiscal 2018 reflectsreductions to the premiu
ms that we pay to the third-party underwriter.In fiscal 2019, Domestic segm
ent online revenue of $6.5  billion increased 10.5% on a comparable basis, primarily due to higher conversion rates and increasedtraffic. As a percentage of
 total Domestic revenue, online revenue increased to 16.6%  versus 15.5% last year.30
----------------------------------------------------------------------------------------------------
Id: MTQxMw==
Description: BESTBUY
Text: Table of ContentsOur operating income incr
eased $57 million  and our operating income as  a percent of revenue remained flat in fiscal 2019 compared to fiscal 2018, primarily d

## Perform a Semantic Hybrid Search

In [22]:
# Semantic Hybrid Search

search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="Embedding")

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["Id", "Text"],
    query_type=QueryType.SEMANTIC, semantic_configuration_name='my-semantic-config', query_caption=QueryCaptionType.EXTRACTIVE, query_answer=QueryAnswerType.EXTRACTIVE,
    top=3
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"Id: {result['Id']}")      
    print('-'*100)  

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")


HttpResponseError: (FeatureNotSupportedInService) Semantic search is not enabled for this service.
Parameter name: queryType
Code: FeatureNotSupportedInService
Message: Semantic search is not enabled for this service.
Parameter name: queryType
Exception Details:	(SemanticQueriesNotAvailable) Semantic search is not enabled for this service.
	Code: SemanticQueriesNotAvailable
	Message: Semantic search is not enabled for this service.

## Integrate whit Langhcin QA RAG-Chat!!!

https://github.com/microsoft/Form-Recognizer-Toolkit/blob/main/SampleCode/Python/sample_rag_langchain.ipynb 